In [1]:
import codecs
import pandas as pd
import os
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder
from os import listdir
from os.path import isfile, join

C:\Users\user\Anaconda3\lib\site-packages\pandas\compat\_optional.py:124: UserWarning: Pandas requires version '1.2.1' or newer of 'bottleneck' (version '1.2.0' currently installed).
  warnings.warn(msg, UserWarning)


In [2]:
input_dir = 'families'
input_files = [f for f in listdir(input_dir) if isfile(join(input_dir, f))]

transactions = list()
for input_file in input_files:
    transaction = list()
    #if input_file != 'F00040 adressage_Nm.txt':
        #continue
    with codecs.open(join(input_dir, input_file), 'r', encoding='utf-8') as f:
        for line_num, line in enumerate(f):
            if line_num >= 2:
                line_elements = line.replace('\n','').replace(' ','').split('\t')
                
                '''orientation = line_elements[21]
                if orientation == 'as2de' or orientation == 'as2des' or orientation == 'de2as' or orientation == 'des2as':
                    orientation = 'direct'
                complexity = line_elements[19]'''
                lex = list()
                lex.append(line_elements[8] + '_' + line_elements[14])
                lex.append(line_elements[10] + '_' + line_elements[17])
                if line_elements[21] == 'as2de' or line_elements[21] == 'as2des':
                    item = lex[0] + ' - ' + line_elements[19] + ' - ' + lex[1]
                elif line_elements[21] == 'de2as' or line_elements[21] == 'des2as':
                    item = lex[1] + ' - ' + line_elements[19] + ' - ' + lex[0]
                else:
                    lex.sort()
                    item = lex[0] + ' - ' + line_elements[19] + '_' + line_elements[21] + ' - ' + lex[1]
                transaction.append(item)
                lex.clear()
    transactions.append(transaction)
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df = pd.DataFrame(te_ary, columns=te.columns_)
df.index = input_files
df

,Adj_X - accidentel - V_enX,Adj_X - motiv-form - Adj_Xistique,Adj_X - motiv-form - Adj_interX,Adj_X - motiv-form - V_Xiser,Adj_X - motiv-sem - Adj_Xique,Adj_X - motiv-sem - Adj_Xétique,Adj_X - motiv-sem - Adj_orthoXaire,Adj_X - motiv-sem - Nm_antiXaire,Adj_X - motiv-sem - V_Xaliser,Adj_X - motiv-sem_indirect - Adj_Xique,...,V_X - simple - Nm_Xing,V_X - simple - Nm_Xion,V_X - simple - Nm_Xis,V_X - simple - Nm_Xière,V_X - simple - Nm_Xment,V_X - simple - Nm_Xon,V_X - simple - Nm_Xoyer,V_X - simple - Nmp_Xment,V_X - simple - V_enX,V_X - simple - V_inX
F00000 abaisser_V.txt,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
F00001 abandonnateur_Nm.txt,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
F00002 abattage_Nm.txt,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
F00003 abcédation_Nf.txt,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
F00004 abîme_Nm.txt,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
F12133 volubilaire_Nf.txt,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
F12134 volvaire_Nf.txt,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
F12135 vulnéraire_Nf.txt,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
F12136 wikiversitaire_Nf.txt,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [7]:
frequent_itemsets = apriori(df, min_support=0.03, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets['# of families'] = frequent_itemsets['support'].apply(lambda x: x*df.shape[0])
frequent_itemsets

,support,itemsets,length,# of families
0,0.035344,(Adj_Xif - simple_indirect - Nf_Xion),1,429.0
1,0.034273,(Nf_X - simple - Adj_Xaire),1,416.0
2,0.033861,(Nf_X - simple - Adj_Xier),1,411.0
3,0.040699,(Nf_X - simple - Adj_Xif),1,494.0
4,0.153650,(Nf_X - simple - Adj_Xique),1,1865.0
...,...,...,...,...
103,0.035920,"(V_X - simple - Nm_Xage, Nm_Xage - simple_indi...",5,436.0
104,0.037568,"(V_X - simple - Nm_Xage, V_X - simple - Nf_Xeu...",5,456.0
105,0.035755,"(V_X - simple - Nm_Xage, Nm_Xage - simple_indi...",5,434.0
106,0.036826,"(V_X - simple - Nm_Xage, Nm_Xage - simple_indi...",5,447.0


In [14]:
rules = association_rules(frequent_itemsets, metric='confidence', min_threshold=0.98)
#rules['antecedent length'] = rules['antecedents'].apply(lambda x: len(x))
#rules['consequent length'] = rules['consequents'].apply(lambda x: len(x))
rules.drop(columns=['antecedent support', 'consequent support', 'lift', 'leverage', 'conviction'], inplace=True)
rules

,antecedents,consequents,support,confidence
0,(Nf_Xeuse - simple_indirect - Nm_Xage),(V_X - simple - Nf_Xeuse),0.041440,0.982422
1,(Nf_Xeuse - simple_indirect - Nm_Xage),(V_X - simple - Nm_Xage),0.041770,0.990234
2,(Nm_Xage - simple_indirect - Nm_Xeur),(V_X - simple - Nm_Xage),0.061789,0.985545
3,(Nm_Xage - simple_indirect - Nm_Xeur),(V_X - simple - Nm_Xeur),0.061789,0.985545
4,(Nm_Xage - simple_indirect - Nm_Xment),(V_X - simple - Nm_Xage),0.032707,1.000000
...,...,...,...,...
162,"(V_X - simple - Nm_Xage, Nf_Xeuse - simple_ind...","(V_X - simple - Nm_Xeur, V_X - simple - Nf_Xeu...",0.035755,0.995413
163,"(Nf_Xeuse - simple_indirect - Nm_Xage, Nf_Xeus...","(V_X - simple - Nm_Xage, V_X - simple - Nm_Xeu...",0.035755,0.990868
164,"(Nf_Xeuse - simple_indirect - Nm_Xage, V_X - s...","(V_X - simple - Nm_Xage, V_X - simple - Nm_Xeu...",0.035755,1.000000
165,"(Nf_Xeuse - simple_indirect - Nm_Xage, V_X - s...","(V_X - simple - Nm_Xage, V_X - simple - Nf_Xeu...",0.035755,0.995413


In [15]:
families_ant_con = list()
families_ant = list()
for chosen_rule_id in rules.index:
    set_of_antecedents = rules.loc[chosen_rule_id, 'antecedents']
    set_of_consequents = rules.loc[chosen_rule_id, 'consequents']
    conditions_ant = list()
    conditions_ant_con = list()
    for ante in set_of_antecedents:
        conditions_ant.append(df[ante])
        conditions_ant_con.append(df[ante])
    for cons in set_of_consequents:
        conditions_ant.append(~df[cons])
        conditions_ant_con.append(df[cons])

    joined_ant = pd.Series([True] * df.shape[0], index=df.index)
    for c in conditions_ant:
        joined_ant &= c
    indices = df.loc[joined_ant].index
    ret_str = str(len(indices)) + ' families: '
    for ind in indices:
        ret_str += ind.split(' ')[0] + ' '
    families_ant.append(ret_str)

    joined_ant_con = pd.Series([True] * df.shape[0], index=df.index)
    for c in conditions_ant_con:
        joined_ant_con &= c
    indices = df.loc[joined_ant_con].index
    ret_str = str(len(indices)) + ' families: '
    for ind in indices:
        ret_str += ind.split(' ')[0] + ' '
    families_ant_con.append(ret_str)
rules['families with ante & cons'] = families_ant_con
rules['families with ante only'] = families_ant
rules

,antecedents,consequents,support,confidence,families with ante & cons,families with ante only
0,(Nf_Xeuse - simple_indirect - Nm_Xage),(V_X - simple - Nf_Xeuse),0.041440,0.982422,503 families: F00002 F00020 F00025 F00050 F000...,9 families: F00141 F01064 F01417 F02138 F03317...
1,(Nf_Xeuse - simple_indirect - Nm_Xage),(V_X - simple - Nm_Xage),0.041770,0.990234,507 families: F00002 F00020 F00025 F00050 F000...,5 families: F03317 F04182 F05084 F05883 F06170
2,(Nm_Xage - simple_indirect - Nm_Xeur),(V_X - simple - Nm_Xage),0.061789,0.985545,750 families: F00007 F00020 F00021 F00024 F000...,11 families: F00758 F03439 F05491 F05779 F0588...
3,(Nm_Xage - simple_indirect - Nm_Xeur),(V_X - simple - Nm_Xeur),0.061789,0.985545,750 families: F00007 F00020 F00021 F00024 F000...,11 families: F03439 F05491 F05779 F05883 F0611...
4,(Nm_Xage - simple_indirect - Nm_Xment),(V_X - simple - Nm_Xage),0.032707,1.000000,397 families: F00002 F00008 F00020 F00021 F000...,0 families:
...,...,...,...,...,...,...
162,"(V_X - simple - Nm_Xage, Nf_Xeuse - simple_ind...","(V_X - simple - Nm_Xeur, V_X - simple - Nf_Xeu...",0.035755,0.995413,434 families: F00020 F00025 F00050 F00052 F000...,0 families:
163,"(Nf_Xeuse - simple_indirect - Nm_Xage, Nf_Xeus...","(V_X - simple - Nm_Xage, V_X - simple - Nm_Xeu...",0.035755,0.990868,434 families: F00020 F00025 F00050 F00052 F000...,2 families: F05883 F06170
164,"(Nf_Xeuse - simple_indirect - Nm_Xage, V_X - s...","(V_X - simple - Nm_Xage, V_X - simple - Nm_Xeu...",0.035755,1.000000,434 families: F00020 F00025 F00050 F00052 F000...,0 families:
165,"(Nf_Xeuse - simple_indirect - Nm_Xage, V_X - s...","(V_X - simple - Nm_Xage, V_X - simple - Nf_Xeu...",0.035755,0.995413,434 families: F00020 F00025 F00050 F00052 F000...,0 families:


In [16]:
#rules.sort_values(by=['confidence'], ascending=False, inplace=True)
rules.to_csv('rules.csv', sep='\t')